#### **1. Nutritional planning**
##### **a) *insert title***

##### **b) *insert title***

##### **c) *insert title***

##### **d) *insert title***

##### **e) *insert title***

#### **2. Transport problem**
##### **a) Linear optimization problem**

Let $x_{ij}$ be the quantity of units transported from source $i \ (S_i)$ to destination $j \ (D_j)$, where $i \in \{1, 2, 3\}$ and $j \in \{1, 2, 3, 4\}$.

**Objective function**

The goal is to minimize the total transportation cost $Z$:
$$\text{Minimize} \ Z = \sum_{i=1}^3 \sum_{j=1}^4 c_{ij}x_{ij}$$
Explicitly using the given costs $(c_{ij})$: $$\text{Min } Z = 10x_{11} + 0x_{12} + 20x_{13} + 11x_{14} + 12x_{21} + 7x_{22} + 9x_{23} + 20x_{24} + 0x_{31} + 14x_{32} + 16x_{33} + 18x_{34}$$

**Constraints**
1. The total volume transported from each source must not exceed the available supply ($A_i$): 
$$\sum_{j=1}^4 x_{ij} \le A_i \ \text{for} \ i = 1, 2, 3$$
- S1: $x_{11} + x_{12} + x_{13} + x_{14} \leq 20$
- S2: $x_{21} + x_{22} + x_{23} + x_{24} \leq 25$
- S3: $x_{31} + x_{32} + x_{33} + x_{34} \leq 15$

2. The total volume transported to each destination must equal the required demand ($V_j$):
$$\sum_{i=1}^3 x_{ij} = V_j \ \text{for} \ j = 1, 2, 3, 4$$
- D1: $x_{11} + x_{21} + x_{31} = 10$
- D2: $x_{12} + x_{22} + x_{32} = 15$
- D3: $x_{13} + x_{23} + x_{33} = 15$
- D4: $x_{14} + x_{24} + x_{34} = 20$

3. The transported quantities must be non-negative:
$$x_{ij} \ge 0 \ \text{for all} \ i, j$$


##### **b) Cheapest transportation plan**

First, we define the parameters of the transportation problem and initialize the linear programming model using the `pulp` library. 

In [ ]:
import pulp
from pulp import LpProblem, LpMinimize, LpVariable, LpStatus, lpSum, value

# Data Definition
Sources = ['S1', 'S2', 'S3']
Destinations = ['D1', 'D2', 'D3', 'D4']

# Cost matrix
costs = {
    'S1': {'D1': 10, 'D2': 0, 'D3': 20, 'D4': 11}, 
    'S2': {'D1': 12, 'D2': 7, 'D3': 9, 'D4': 20}, 
    'S3': {'D1': 0, 'D2': 14, 'D3': 16, 'D4': 18}
}

# Supply
supply = {'S1': 20,'S2': 25,'S3': 15}

# Demand
demand = {'D1': 10, 'D2': 15, 'D3': 15, 'D4': 20}

# Initializing the problem as a minimization problem
model = pulp.LpProblem("Transportation_Optimization", pulp.LpMinimize)

# Defining the decision variables 
x = pulp.LpVariable.dicts("x", 
                          ((i, j) for i in Sources for j in Destinations), 
                          lowBound=0,
                          cat= 'Continuous')



Next, we formulate the objective function and add the supply and demand constraints to the model, followed by solving the problem. 

In [2]:
# Objective function: minimize total cost (Z = sum(c_ij * x_ij))
model += pulp.lpSum(costs [i][j] * x[(i, j)] for i in Sources for j in Destinations), "Total_Transportation_Cost"

# Supply constraints (total flow out of Source i <= Supply A_i)
for i in Sources: 
    model += pulp.lpSum(x[(i, j)] for j in Destinations) <= supply[i], f"Supply_Constraint_{i}"

# Demand constraints (Total flow into Destination j = Demand V_j)
for j in Destinations:
    model += pulp.lpSum(x[(i, j)] for i in Sources) == demand[j], f"Demand_Constraint_{j}"

# Solving the problem
model.solve()

# Checking the status of the solution
print(f"Solver Status: {pulp.LpStatus[model.status]}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/25/0hf3c7d931q0rktyfwqp3g_w0000gn/T/dc7ee6e71f774d40a7e1c41d98eb2dec-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/25/0hf3c7d931q0rktyfwqp3g_w0000gn/T/dc7ee6e71f774d40a7e1c41d98eb2dec-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 47 RHS
At line 55 BOUNDS
At line 56 ENDATA
Problem MODEL has 7 rows, 12 columns and 24 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 7 (0) rows, 12 (0) columns and 24 (0) elements
0  Obj 0 Primal inf 60 (4)
6  Obj 460
Optimal - objective value 460
Optimal objective 460 - 6 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.02

Solver Status: Opt

The problem was successfully solved. The optimal solution yields the minimum total cost and the optimal flow plan, which respects all supply and demand limitations. We can now print the final answer, which is the minimal cost and allocation of goods for each route. 

In [3]:
# Minimum total costs
if pulp.LpStatus[model.status] == "Optimal":
    min_cost = pulp.value(model.objective) 
    print("\n==================================================")
    print (f"The minimum required transportation cost (Z) is: €{min_cost:,.2f}")
    print("==================================================")

    # Optimal Transport Plan (Flows > 0)
    print("\nOptimal Transportation Plan (Flows > 0):")

    # A header for the table output
    print("{:<15} {:<15} {:<10}" .format("Source", "Destination", "Units Sent")) 
    print("-" * 40)

    for i in Sources: 
        for j in Destinations: 
            amount = pulp.value(x[(i,j)])
            if amount > 0:
                print("{:<15}" "{:<15}" "{:<10.0f}" .format(i, j, amount))
else:
    print("\nError: The optimization problem could not be solved. Status: {pulp.LpStatus[model.status]}")




The minimum required transportation cost (Z) is: €460.00

Optimal Transportation Plan (Flows > 0):
Source          Destination     Units Sent
----------------------------------------
S1             D2             5         
S1             D4             15        
S2             D2             10        
S2             D3             15        
S3             D1             10        
S3             D4             5         


##### **c) Fixed costs**

We introduce a binary variable $y_{ij}$ to track whether a route is active. 
1. New decision variable: $y_{ij} \in \{0,1\}$: Equals 1 if goods are transported from source $i$ to destination $j$, and 0 otherwise. 
2. Objective function: the goal is to minimize the sum of variable costs and fixed costs. 
$$\text{Minimize } Z = \left( \sum_{i=1}^{3} \sum_{j=1}^{4} c_{ij} x_{ij} \right) + \left( \sum_{i=1}^{3} \sum_{j=1}^{4} 100 \cdot y_{ij} \right)$$
3. The binary variable $(y_{ij})$ is linked to the quantity $(x_{ij})$, using $M = 60$ (total supply/demand):
$$x_{ij} \le 60 \cdot y_{ij} \ \text{for all} \ i, j$$

In this code we add the binary variables, update the objective function and add the Big M constraints: 

In [11]:
# Data Definition
Sources = ['S1', 'S2', 'S3']
Destinations = ['D1', 'D2', 'D3', 'D4']

# Cost matrix
costs = {
    'S1': {'D1': 10, 'D2': 0, 'D3': 20, 'D4': 11}, 
    'S2': {'D1': 12, 'D2': 7, 'D3': 9, 'D4': 20}, 
    'S3': {'D1': 0, 'D2': 14, 'D3': 16, 'D4': 18}
}

M = 60
FIXED_COST = 100

# New model
model = pulp.LpProblem("ILP_with_Fixed_Costs", pulp.LpMinimize)

# Defining the binary variables
y = pulp.LpVariable.dicts("y",
                          ((i, j) for i in Sources for j in Destinations), 
                          cat='Binary') 
x = pulp.LpVariable.dicts("x",
                          ((i, j) for i in Sources for j in Destinations),
                          lowBound=0, cat='Continuous')

# Supply and demand constraints
for i in Sources:
    model += pulp.lpSum(x[(i,j)] for j in Destinations) <= supply[i], f"Supply_{i}"

for j in Destinations:
    model += pulp.lpSum(x[(i,j)] for i in Sources) >= demand[j], f"Demand_{j}"

# Redefining the objective
model += (pulp.lpSum(costs[i][j] * x[(i, j)] for i in Sources for j in Destinations)
          + pulp.lpSum(FIXED_COST * y[(i, j)] for i in Sources for j in Destinations)), "ILP_Total_Cost_with_Fixed"

# Adding in the constraints
for i in Sources:
    for j in Destinations: 
        model += x[(i, j)] <= M * y[(i, j)], f"M_Constraint_{i}_{j}"

# Solving the model
model.solve()

# Displaying the results
print("\n=== ILP SOLUTION (Fixed Costs) ===")
print(f"Solver Status: {pulp.LpStatus[model.status]}")

if pulp.LpStatus[model.status] == "Optimal":
    ilp_min_cost = pulp.value(model.objective)
    ilp_fixed_cost = pulp.value(pulp.lpSum(FIXED_COST * y[(i, j)] for i in Sources for j in Destinations))

    print("\n==================================================")
    print(f"Minimum total cost including fixed charges: €{ilp_min_cost:,.2f}")
    print(f"Fixed cost component: €{ilp_fixed_cost:,.2f}")
    print("==================================================")
    
    print("\nOptimal ILP Transport Plan (Flows > 0):")
    print("{:<15} {:<15} {:<10} {:<10}".format("Source", "Destination", "Units Sent", "Fixed Used")) 
    print("-" * 55)

    ilp_routes_used = 0
    for i in Sources:
        for j in Destinations:
            amount = pulp.value(x[(i, j)])
            is_used = pulp.value(y[(i, j)])
            
            if is_used > 0.5:
                ilp_routes_used += 1
                print("{:<15} {:<15} {:<10.0f} {:<10.0f}".format(i, j, amount, is_used))

    print(f"\nTotal number of distinct routes used: {ilp_routes_used}")



Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/25/0hf3c7d931q0rktyfwqp3g_w0000gn/T/117a2d6c094d41fd90d8ab49ad0ffeac-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/25/0hf3c7d931q0rktyfwqp3g_w0000gn/T/117a2d6c094d41fd90d8ab49ad0ffeac-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 24 COLUMNS
At line 119 RHS
At line 139 BOUNDS
At line 152 ENDATA
Problem MODEL has 19 rows, 24 columns and 48 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 560 - 0.00 seconds
Cgl0004I processed model has 19 rows, 24 columns (12 integer (12 of which binary)) and 48 elements
Cbc0038I Initial state - 6 integers unsatisfied sum - 1.96667
Cbc0038I Pass   1: suminf.    0.73333 (2) obj. 868.333 iterations 4
Cbc0038I Solution found of 995
Cbc

The introduction of fixed costs compels the solver to adopt a different optimization strategy. The ILP solution (€850.00) is significantly higher due to the €400.00 in fixed costs (4 routes $\times$ €100). The solver prioritizes consolidating the total shipment onto a smaller number of distinct routes (4), even if it means using routes that have slightly higher variable costs (e.g., $c_{ij}$) than the minimum possible 6 routes used in the pure LP model. The goal shifts from finding the absolute cheapest per-unit cost to finding the cheapest set of routes to open.

#### **3. Inventory managment**
##### **a) *insert title***

##### **b) *insert title***

##### **c) *insert title***

#### **4. Single-Machine Scheduling**
##### **a) *insert title***

##### **b) *insert title***

##### **c) *insert title***

##### **d) *insert title***